In [ ]:
import numpy as np

def givens_rotation(v1, v2):
    """
    计算Givens旋转参数。
    """
    t = np.sqrt(v1**2 + v2**2)
    if t == 0:
        cs = 1
        sn = 0
    else:
        cs = v1 / t
        sn = v2 / t
    return cs, sn

def apply_givens_rotation(h, cs, sn, k):
    """
    将Givens旋转应用于Hessenberg矩阵的第k列。
    """
    for i in range(k - 1):
        temp = cs[i] * h[i] + sn[i] * h[i+1]
        h[i+1] = -sn[i] * h[i] + cs[i] * h[i+1]
        h[i] = temp
    
    cs_k, sn_k = givens_rotation(h[k-1], h[k])
    
    # 消除 H(k+1, k)
    h[k-1] = cs_k * h[k-1] + sn_k * h[k]
    h[k] = 0.0
    
    return h, cs_k, sn_k

def arnoldi(A, Q, k):
    """
    Arnoldi方法生成新的Krylov向量和Hessenberg矩阵的列。
    """
    q = A @ Q[:, k-1]  # 注意Python中索引从0开始
    
    h = np.zeros(k + 1)
    for i in range(k):
        h[i] = Q[:, i].conj() @ q
        q = q - h[i] * Q[:, i]
        
    h[k] = np.linalg.norm(q)
    if h[k] == 0:
        return h, q  # 防止除以零
    q = q / h[k]
    return h, q

def gmres(A, b, x, max_iterations, threshold):
    """
    广义最小残差方法 (GMRES)
    """
    n = A.shape[0]
    m = max_iterations
    
    r = b - A @ x
    b_norm = np.linalg.norm(b)
    error = np.linalg.norm(r) / b_norm
    
    if error <= threshold:
        return x, np.array([error])
    
    sn = np.zeros(m)
    cs = np.zeros(m)
    
    e1 = np.zeros(m + 1)
    e1[0] = 1
    
    e = [error]
    
    r_norm = np.linalg.norm(r)
    Q = np.zeros((n, m + 1), dtype=A.dtype)
    Q[:, 0] = r / r_norm
    
    beta = r_norm * e1
    
    H = np.zeros((m + 1, m), dtype=A.dtype)
    
    for k in range(m):
        # 运行Arnoldi
        h_col, q_new = arnoldi(A, Q, k + 1)
        H[:k+2, k] = h_col
        Q[:, k+1] = q_new
        
        # 应用Givens旋转
        # 注意Python的索引和切片
        H[:k+2, k], cs[k], sn[k] = apply_givens_rotation(H[:k+2, k], cs, sn, k + 1)
        
        # 更新残差向量
        beta[k + 1] = -sn[k] * beta[k]
        beta[k] = cs[k] * beta[k]
        
        error = abs(beta[k+1]) / b_norm
        e.append(error)
        
        if error <= threshold:
            break
            
    # 计算结果
    # 调整H和beta的大小以匹配
    H_k = H[:k+1, :k+1]
    beta_k = beta[:k+1]
    
    y = np.linalg.solve(H_k, beta_k)
    x = x + Q[:, :k+1] @ y
    
    return x, np.array(e)

---
### 广义最小残差法（GMRES）原理

GMRES（Generalized Minimal Residual Method）是一种用于求解大型非对称线性方程组 $Ax=b$ 的迭代算法。它的核心思想是在一个不断扩展的Krylov子空间中，寻找一个近似解，使得残差向量 $r_k = b - Ax_k$ 的**范数最小化**。

#### 1. GMRES的目标

与直接方法（如高斯消元法）不同，GMRES通过一系列迭代来逐步逼近精确解。在第 $k$ 步迭代中，GMRES的目标是在一个 $k$ 维子空间中找到一个向量 $x_k$，使得以下残差范数最小：

$$\min_{x_k} \Vert b - Ax_k \Vert_2$$

#### 2. Krylov子空间

GMRES的关键在于如何构建这个用于寻找近似解的子空间。它使用的是**Krylov子空间** $K_k$，该子空间由以下向量张成：

$$K_k = \text{span}\{r_0, Ar_0, A^2r_0, \dots, A^{k-1}r_0\}$$

其中 $r_0 = b - Ax_0$ 是初始残差（通常 $x_0=0$）。

#### 3. Arnoldi算法的作用

直接使用上述基向量可能导致数值不稳定，因为它们会变得近似线性相关。因此，GMRES利用**Arnoldi算法**来生成一个**正交基** $Q_k = \{q_1, q_2, \dots, q_k\}$ 来代替。

Arnoldi算法的迭代过程可以总结为以下关系式：

$$AQ_k = Q_{k+1} \bar{H}_k$$

* $Q_k$：由Arnoldi算法生成的**正交基**，其列向量是Krylov子空间的正交基。
* $\bar{H}_k$：一个 $(k+1) \times k$ 的**上Hessenberg矩阵**，它包含了Arnoldi过程中计算出的所有内积和范数。

#### 4. 最小化残差

现在，我们把解 $x_k$ 表示为正交基 $Q_k$ 的线性组合：

$$x_k = x_0 + Q_k y_k$$

其中 $y_k$ 是一个 $k$ 维系数向量。将此表达式代入残差方程：

$$r_k = b - A(x_0 + Q_k y_k) = (b - Ax_0) - AQ_k y_k = r_0 - AQ_k y_k$$

我们知道 $r_0$ 的范数是 $\beta = \Vert r_0 \Vert_2$，并且 $r_0$ 可以写成 $\beta q_1$（因为 $q_1$ 是 $r_0$ 的单位化向量）。结合Arnoldi关系式，残差方程变为：

$$r_k = \beta q_1 - Q_{k+1} \bar{H}_k y_k$$

由于 $Q_{k+1}$ 的列向量是正交的，我们最小化残差范数的问题就等价于：

$$\min_{y_k} \Vert \beta e_1 - \bar{H}_k y_k \Vert_2$$

其中 $e_1$ 是一个 $(k+1)$ 维的单位向量。这是一个简单的**最小二乘问题**，可以通过**QR分解**或**Givens旋转**来高效求解。

#### 5. Givens旋转的应用

在您的代码中，Givens旋转被用来对 $\bar{H}_k$ 进行**QR分解**。在每次Arnoldi迭代后，新的 $\bar{H}_k$ 矩阵会扩展一列。Givens旋转的作用是逐步消除新列中的次对角线元素，从而将 $\bar{H}_k$ 转化为一个上三角矩阵 $R_k$。

同时，这个旋转也作用于右侧的向量 $\beta e_1$。最终，最小二乘问题被简化为求解一个小的上三角方程组：

$$R_k y_k = g_k$$

其中 $g_k$ 是通过一系列Givens旋转得到的向量。

#### 6. 算法总结

GMRES算法的每一步可以概括为：

1.  **Arnoldi迭代**：从上一步的 $q_k$ 生成新的正交向量 $q_{k+1}$，并更新Hessenberg矩阵 $\bar{H}_k$。
2.  **Givens旋转**：对 $\bar{H}_k$ 的新列应用Givens旋转，以保持矩阵的上三角形式，同时更新残差向量 $\beta e_1$。
3.  **计算残差**：新的残差范数可以直接从更新后的 $\beta$ 向量中得到。如果满足收敛条件，则停止。
4.  **求解**：在停止时，求解一个小型的上三角方程组 $R_k y_k = g_k$ 来得到系数向量 $y_k$。
5.  **更新解**：用 $y_k$ 来更新近似解 $x_k = x_0 + Q_k y_k$。

GMRES的优点在于它能处理非对称矩阵，并且在每一步迭代中残差范数都是单调递减的，保证了收敛性。然而，它需要存储所有的 $Q_k$ 和 $\bar{H}_k$ 向量，因此在大规模问题上可能存在内存问题，这也是“隐式重启”方法（IRAM）出现的原因。